# 加上两个用户行为特征

1. lb8699基础上
2. 添加他的新特征



In [23]:
import sys
sys.path.append("..")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline
%run ../util/base_frame.py

train_df = pd.read_table('../../round1_ijcai_18_train_20180301.txt',sep=' ')
#test_df = pd.read_table('../../round1_ijcai_18_test_a_20180301.txt',sep=' ')
test_df = pd.read_table('../../round1_ijcai_18_test_b_20180418.txt',sep=' ')

fa = BaseFrame(train_df, test_df, 6)

# 1 先处理时序数据, 基本标签化处理

In [24]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数
# date最终不使用，直接用day(第 0 - 7 天)

from sklearn import preprocessing
from datetime import datetime
le = preprocessing.LabelEncoder()

fa.df['weekday'] = fa.df['context_timestamp'].apply(lambda x: datetime.fromtimestamp(x).weekday())
fa.df['hour'] = fa.df['context_timestamp'].apply(lambda x: datetime.fromtimestamp(x).hour)
fa.df['minute'] = fa.df['context_timestamp'].apply(lambda x: datetime.fromtimestamp(x).minute)
fa.df['seconds'] = fa.df['context_timestamp'].apply(lambda x: datetime.fromtimestamp(x).second)
print(fa.df.shape)

(521026, 34)


In [25]:
# 标签化
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()

for col in ['item_id', 'item_brand_id', 'item_city_id', 'shop_id', 'user_id']:
    fa.df[col] = lbl.fit_transform(fa.df[col])

# 2 特征处理

### 时间交易频次特征

In [26]:
def groupbySizeFeat(df, feats, name):
    target = df.groupby(feats).size().reset_index().rename(columns={0: name})
    return pd.merge(df, target, 'left', on=feats)

fa.df = groupbySizeFeat(fa.df, ['user_id', 'day'], 'user_query_day')
fa.df = groupbySizeFeat(fa.df, ['user_id', 'weekday'], 'user_query_weekday')
fa.df = groupbySizeFeat(fa.df, ['user_id', 'day', 'hour'], 'user_query_day_hour')
fa.df = groupbySizeFeat(fa.df, ['day', 'user_id', 'item_id'], 'day_user_item_id')
fa.df = groupbySizeFeat(fa.df, ['day', 'hour', 'minute', 'user_id', 'item_id'], 'day_hour_minute_user_item_id')
fa.df = groupbySizeFeat(fa.df, ['day', 'hour', 'item_id'], 'number_day_hour_item_id')
fa.df = groupbySizeFeat(fa.df, ['item_id', 'user_id'], 'item_user_id')
fa.df = groupbySizeFeat(fa.df, ['item_category_list', 'item_city_id'], 'item_category_city_id')
fa.df = groupbySizeFeat(fa.df, ['item_category_list', 'item_sales_level'], 'item_category_sales_level')
fa.df = groupbySizeFeat(fa.df, ['item_category_list', 'item_price_level'], 'item_category_price_level')

fa.df = groupbySizeFeat(fa.df, ['item_id', 'item_sales_level'], 'item_ID_sales_level')
fa.df = groupbySizeFeat(fa.df, ['item_id', 'item_collected_level'], 'item_ID_collected_level')
fa.df = groupbySizeFeat(fa.df, ['user_id'], 'number_user_id')
fa.df = groupbySizeFeat(fa.df, ['shop_id'], 'number_shop_id')
print(fa.df.shape)

(521026, 48)


### 类目特征OneHotEncoding

In [27]:
for i in range(5):
    fa.df['predict_category_property' + str(i)] = lbl.fit_transform(fa.df['predict_category_property'].map(
        lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))

for i in range(1, 3):
    fa.df['item_category_list' + str(i)] = lbl.fit_transform(fa.df['item_category_list'].map(
        lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))  # item_category

for i in range(10):
    fa.df['item_property_list' + str(i)] = lbl.fit_transform(fa.df['item_property_list'].map(
        lambda x: str(str(x).split(';')[i]) if len(str(x).split(';')) > i else ''))


In [28]:
fa.df['gender0'] = fa.df['user_gender_id'].apply(lambda x: x + 1 if x == -1 else x)
fa.df['age0'] = fa.df['user_age_level'].apply(lambda x: 1003 if x == -1  else x)
fa.df['occupation0'] = fa.df['user_occupation_id'].apply(lambda x: 2005 if x == -1  else x)
fa.df['star0'] = fa.df['user_star_level'].apply(lambda x: 3006 if x == -1 else x)

In [29]:
fa.df = groupbySizeFeat(fa.df, ['item_brand_id', 'shop_review_positive_rate'], 'number_item_brand_positive_rate')
fa.df = groupbySizeFeat(fa.df, ['item_brand_id', 'shop_star_level'], 'number_item_brand_shop_star')
fa.df = groupbySizeFeat(fa.df, ['item_city_id', 'item_pv_level'], 'number_item_city_pv_level')
fa.df = groupbySizeFeat(fa.df, ['item_city_id', 'user_id'], 'number_item_city_user_id')
fa.df = groupbySizeFeat(fa.df, ['item_price_level', 'item_sales_level'], 'number_item_price_sales_level')
fa.df = groupbySizeFeat(fa.df, ['predict_category_property', 'item_sales_level'], 'number_predict_category_sales_level')
fa.df = groupbySizeFeat(fa.df, ['item_collected_level', 'shop_id'], 'number_collected_shop_id')


### 访问频次
>比如说user_id, shop_id group: user访问shop的第几次

In [30]:
%run ../util/time_utils.py


fa.df['last_1_user_shop'] = getLastImpressTime(fa.df, ['user_id', 'shop_id'])
fa.df['last_1_user_item'] = getLastImpressTime(fa.df, ['user_id', 'item_id'])
fa.df['last_1_user_shop_item'] = getLastImpressTime(fa.df, ['user_id', 'shop_id', 'item_id'])
print(fa.df.shape)

(521026, 79)


In [31]:
%run ../util/time_utils.py

fa.df['next_1_user_shop'] = getNextImpressTime(fa.df, ['user_id', 'shop_id'])
fa.df['next_1_user_item'] = getNextImpressTime(fa.df, ['user_id', 'item_id'])
fa.df['next_1_user_shop_item'] = getNextImpressTime(fa.df, ['user_id', 'shop_id', 'item_id'])

### 重复列特征

In [32]:
%run ../util/time_utils.py
dup_feat = ['item_id', 'user_id']
fa.df = generateColDupByDay(fa.df, dup_feat, list(range(1, 8)), verbose=False)
print(fa.df.shape)

(521026, 96)


### 交易率特征选择

*固定smooth为10*

1. 特征是否独立提升?
2. 平滑是否不会改变正负向?

##### 正向特征

> item_city_id, shop_id, user_gender_id, item_sales_level, item_collected_level, shop_review_num_level

##### 负向特征

> item_price_level,
item_id,item_brand_id,item_pv_level,user_age_level,
user_occupation_id,user_star_level,context_page_id,
shop_review_positive_rate,shop_star_level,
shop_score_service,shop_score_description,day,hour


In [33]:
# 计算前一天的交易率set到下一天，第0天用回自己
%run ../util/trade_info.py

trade_rela = ['item_id', 'item_brand_id', 'shop_id', 'user_id']

generateTradeRateByDate(fa.df, trade_rela, 7, None, verbose=False, glbSmoothing=200, glbMean0=0.05)
print(fa.df.shape)

C:\Anaconda3\lib\site-packages\pandas\core\series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


(521026, 105)


### 复合交易率

In [34]:
fa.df.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level',
       ...
       'user_id_dup_gg_7', '_key1', 'exp_d_item_id_7', 'cnt_d_item_id_7',
       'exp_d_item_brand_id_7', 'cnt_d_item_brand_id_7', 'exp_d_shop_id_7',
       'cnt_d_shop_id_7', 'exp_d_user_id_7', 'cnt_d_user_id_7'],
      dtype='object', length=105)

In [35]:
%run  ../util/trade_info.py


fa.df['u2i_sz'], fa.df['u2i_sum'], fa.df['u2i_rate'] = getMultiTradeRate(fa.df, ['user_id', 'item_id'])
fa.df['u2shop_sz'], fa.df['u2shop_sum'], fa.df['u2shop_rate'] = getMultiTradeRate(fa.df, ['user_id', 'shop_id'])
fa.df['u2brand_sz'], fa.df['u2brand_sum'], fa.df['u2brand_rate'] = getMultiTradeRate(fa.df, ['user_id', 'item_brand_id'])
fa.df['u2page_sz'], fa.df['u2page_sum'], fa.df['u2page_rate'] = getMultiTradeRate(fa.df, ['user_id', 'context_page_id'])
fa.df['u2hour_sz'], fa.df['u2hour_sum'], fa.df['u2hour_rate'] = getMultiTradeRate(fa.df, ['user_id', 'hour'])
fa.df['u2sstar_sz'], fa.df['u2sstar_sum'], fa.df['u2sstar_rate'] = getMultiTradeRate(fa.df, ['user_id', 'shop_star_level'])
print(fa.df.shape)

data before day 0 is setted, with period 7, data size 78268
data before day 1 is setted, with period 7, data size 70931
data before day 2 is setted, with period 7, data size 68387
data before day 3 is setted, with period 7, data size 71199
data before day 4 is setted, with period 7, data size 68318
data before day 5 is setted, with period 7, data size 63614
data before day 6 is setted, with period 7, data size 57421
data before day 7 is setted, with period 7, data size 42888
data before day 0 is setted, with period 7, data size 78268
data before day 1 is setted, with period 7, data size 70931
data before day 2 is setted, with period 7, data size 68387
data before day 3 is setted, with period 7, data size 71199
data before day 4 is setted, with period 7, data size 68318
data before day 5 is setted, with period 7, data size 63614
data before day 6 is setted, with period 7, data size 57421
data before day 7 is setted, with period 7, data size 42888
data before day 0 is setted, with period

### 复合类型拆解

In [36]:
fa.df.columns.values

array(['instance_id', 'item_id', 'item_category_list',
       'item_property_list', 'item_brand_id', 'item_city_id',
       'item_price_level', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id',
       'predict_category_property', 'shop_id', 'shop_review_num_level',
       'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade', 'date', 'day', 'data_set',
       'weekday', 'hour', 'minute', 'seconds', 'user_query_day',
       'user_query_weekday', 'user_query_day_hour', 'day_user_item_id',
       'day_hour_minute_user_item_id', 'number_day_hour_item_id',
       'item_user_id', 'item_category_city_id',
       'item_category_sales_level', 'item_category_price_level',
       'item_ID_sales_level', 'item_ID_collected_level', 'number_

In [37]:
%run ../util/complex_type.py
from sklearn import preprocessing
from datetime import datetime
le = preprocessing.LabelEncoder()
fa.df = process_complex_types_v2(fa.df, get_icl_map(fa.df), get_ipl_map(fa.df))

fa.df['item_category_1'] = le.fit_transform(fa.df.item_category_1)
fa.df['item_category_2'] = le.fit_transform(fa.df.item_category_2)
print(fa.df.shape)

get_icl_map ... 
get_ipl_map ... 
processing predict_category_property ...
processing item_property_list ...
processing item_category_list ...
generating item_category_1, item_category_2 ...
(521026, 132)


In [38]:
# fa.df['context_timestamp'] = fa.df.context_timestamp_x
# del fa.df['context_timestamp_y']
# del fa.df['context_timestamp_x']
#del fa.df['context_timestamp_y']

In [39]:
%run ../util/complex_type.py

cnt_user_item_review = fa.df[["user_id", "item_id", "instance_id"]].groupby(["user_id", "item_id"])['instance_id'].count().to_dict() 
cnt_user_cate_review = fa.df[["user_id", "item_category_1", "instance_id"]].groupby(["user_id", "item_category_1"])['instance_id'].count().to_dict()  

f1 = set_review_cnt("user_id", "item_id", cnt_user_item_review)
f2 = set_review_cnt("user_id", "item_category_1", cnt_user_cate_review)

tmp = fa.df.sort_values(by="context_timestamp")
tmp["item_review_cnt"] = tmp[["user_id", "item_id"]].apply(f1, axis=1)
tmp["cate_review_cnt"] = tmp[["user_id", "item_category_1"]].apply(f2, axis=1)
fa.df = tmp.sort_index()

print(fa.df.shape)

(521026, 134)


In [40]:
print(fa.df.shape, fa.y_test.shape)

(521026, 134) (42888,)


# 3 fitXgb训练

In [ ]:
##########################
## 刷新base_frame(debug)
# %run ../util/base_frame.py

# tmp = fa.df
# fa = BaseFrame(train_df, test_df, 7)
# fa.df = tmp

In [ ]:
import xgboost
from sklearn.model_selection import ParameterGrid
#import lightgbm as lgb
%run ../util/base_frame.py



non_feat_columns = ['context_timestamp', 'instance_id', 'is_trade', 'context_id',
                   'item_property_list', 'item_category_list',  'predict_category_property','tmp'
                   ]

# cnt_d_shop_id 单特征变差，全集变好
neg_feat_columns = ['exp_d_user_id_7', 'cnt_d_item_id_7', 'cnt_d_item_brand_id_7']



params = {
    'max_depth':[3],
    'n_estimators':[3000],
    'min_child_weight':[5],
    'gamma':[5]
}


m = None
if sum(fa.y_test == -1) == 0:
    # 进入测试阶段
    xgb = xgboost.XGBClassifier(n_jobs=7)
    for g in ParameterGrid(params):
        xgb.set_params(**g)
        print('>', '- '*40, '\n\n> ', g, '\n')
#         fa.fit(xgb, pos_feat_columns, '11', drop=False)
        fa.fit(xgb, non_feat_columns+neg_feat_columns, '14', drop=True, early_stopping_rounds=150)
        print(xgb.best_score)
        # fa.fit(xgb, non_feat_columns+['exp_d_user_id', 'cnt_d_item_brand_id'], '11', drop=True)
else:
    # 进入提交阶段
    xgb = xgboost.XGBClassifier(**{'gamma': 5, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 227, 'n_jobs': 7})
    fa.fit(xgb, non_feat_columns+neg_feat_columns, '20_6917_7967', drop=True, early_stopping_rounds=150)


X_train (478138, 121), y_train (478138,), X_test (42888, 121), y_test (42888,)
> (478138, 121) 



In [ ]:
fig,ax = plt.subplots(figsize=(12,18))
xgboost.plot_importance(xgb, ax=ax, importance_type='cover')
plt.show()